In [ ]:
import gdown

file_id = "1YzxHl_2DVw-396g-e0PxQrCcFZ2aNUHd"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dataset.zip"
gdown.download(url, output, quiet=False)

In [ ]:
import gdown

file_id = "1rX2_7vs7DoprqIvQmMKk8KLLyEdCW4Os"
url = f"https://drive.google.com/uc?id={file_id}"
output = "dataset1.zip"
gdown.download(url, output, quiet=False)

In [ ]:
import zipfile

with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

In [ ]:
import zipfile

with zipfile.ZipFile("dataset1.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset1")

##Labraries##


In [1]:
import string
import os
from  PIL import Image
import numpy as np
import pickle
from tensorflow.keras.applications.xception import Xception,preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import add
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout
import tensorflow as tf
import matplotlib.pyplot as plt


##implementation##

In [ ]:
tokens='/content/caption.token.txt'

def readFile(path):
  file=open(path,'r')
  data=file.read()
  file.close()
  return data


In [ ]:
data=readFile(tokens)

In [ ]:
data=data.split('\n')

In [ ]:

data[0].split('\t')

In [ ]:
def analysis(path):
  data=readFile(path)
  data=data.split('\n')
  descrabtions={}
  for record in data:

    img,caption =record.split('\t')
    if img[:-2] in descrabtions:
      descrabtions[img[:-2]].append(caption)
    else:
      descrabtions[img[:-2]]=[caption]
  return descrabtions

In [ ]:
data=analysis(tokens)

In [ ]:
data

In [ ]:
caption=data['2004674713_2883e63c67.jpg'][0]

In [ ]:
caption

In [ ]:
caption=caption.replace('-','')

In [ ]:
def preprocesscaption(caption):
  caption_list=caption.split()
  punct=str.maketrans('','',string.punctuation)
  caption_list=[word.lower() for word  in caption_list if len(word)>1 and (word.isalpha())]
  caption_list=[word.translate(punct) for word in caption_list]
  caption_list
  return ' '.join(caption_list)

def clean_text(data):
  for img,captions in data.items():
    for index,caption in enumerate(captions):
     data[img][index]=preprocesscaption(caption)
  return data

In [ ]:
data

In [ ]:
clean_captions=clean_text(data)

In [ ]:
clean_captions

In [ ]:
clean_captions['2096771662_984441d20d.jpg']

In [ ]:
def gentrate_repository(data):

  repository_vocab=set()
  repository_vocab.update(['hello','world','hello'])

  repository_vocab

  for img in data.keys():
    for caption in data[img]:
      repository_vocab.update(caption.split())
  return repository_vocab

def write_file(path,data):
   lines=[]
   for img,captions in data.items():
    for caption in captions:
      lines.append(img+'\t'+caption)
   data='\n'.join(lines)
   file=open(path,'w')
   file.write(data)
   file.close()

In [ ]:
repository_vocab=gentrate_repository(clean_captions)


In [ ]:
len(repository_vocab)

In [ ]:
repository_vocab

In [ ]:
write_file('/content/clean_tokens.txt',clean_captions)

In [ ]:
path_images='/content/dataset/images'

os.listdir(path_images)

In [ ]:
imagefile=path_images+'/'+os.listdir(path_images)[2]

In [ ]:
imagefile

In [ ]:
img=Image.open(imagefile)

In [ ]:
model=Xception(include_top=False,pooling='avg')


In [ ]:
def feature_extraction(path,model):
  features={}
  for imageName in os.listdir(path):
    completepath=path_images+'/'+imageName
    img=Image.open(completepath)
    img=img.resize((299,299))
    img=np.expand_dims(img,axis=0)
    img=img/127.5
    img=img-1.0
    feature=model.predict(img)
    features[imageName]=feature

  return features


In [ ]:
#f=feature_extraction(path_images,model)

In [ ]:

#pickle.dump(f,open('/content/imgs_features.bin','wb'))

In [ ]:
f=pickle.load(open('/content/dataset1/imgs_features.bin','rb'))

In [ ]:
f

In [ ]:
train_images_path='/content/caption.trainImages.txt'
clean_tokens_path='/content/clean_tokens.txt'
feature_path='/content/dataset1/imgs_features.bin'

In [ ]:
def loadimages(path):
  data=readFile(path)
  data=data.split('\n')
  return data

def load_tokens(path,images):
  data=readFile(path)
  lines=data.split('\n')
  tokens={}
  for line in lines:
    img,caption=line.split('\t')
    if img in images:
      if img not in tokens:
        tokens[img]=[]
      tokens[img].append('<start> ' +caption+ ' <end>')

  return tokens

def load_features(path,images):
  features=pickle.load(open(path,'rb'))
  selected_features=[{image:features[image]} for image in images  if image in features]
  return selected_features


In [ ]:
train_image=loadimages(train_images_path)

In [ ]:
train_image

In [ ]:
train_tokens=load_tokens(clean_tokens_path,train_image)

In [ ]:
train_tokens

In [ ]:
train_features=load_features(feature_path,train_image)

In [ ]:
train_features

In [ ]:
def fetch_captions(data):

  captionss=[]
  for captions in train_tokens.values():
    for caption in captions:
      captionss.append(caption)
  return captionss

def create_tokenizer(captionss):
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(captionss)
  return tokenizer
def longestcaptions(captionss):
  return max(len(caption.split()) for caption in captionss)

In [ ]:
captionss=fetch_captions(train_tokens)
tokenizer=create_tokenizer(captionss)

In [ ]:
captionss

In [ ]:
tokenizer.word_index

In [ ]:
vocab_size=len(tokenizer.word_index)+1

In [ ]:
vocab_size

In [ ]:
tokenizer.texts_to_sequences([train_tokens['2256320794_0286c31bfa.jpg'][0]])[0]

In [ ]:
max_cap=longestcaptions(captionss)

In [ ]:
max_cap

In [ ]:
seq=train_tokens['2256320794_0286c31bfa.jpg'][0].split()

In [ ]:
seq

In [ ]:
for word_index in range(len(seq)):
  in_seq=seq[:word_index]
  out_w=seq[word_index]
  print(in_seq,out_w)

In [ ]:
def bulid_sequences(tokenizer,max_cap,vocab_size,feature,caption):
  input1,input2,output=[],[], []
  for cap in caption:
    seq=tokenizer.texts_to_sequences([cap])[0]

    for word_index in range(len(seq)):
      in_seq=seq[:word_index]
      in_seq=pad_sequences([in_seq],maxlen=max_cap,padding='post')[0]
      out_w=seq[word_index]
      out_w=to_categorical([out_w],num_classes=vocab_size)[0]
      input1.append(feature)
      input2.append(in_seq)
      output.append(out_w)
  return np.array(input1),np.array(input2),np.array(output)
def datagenrator(tokenizer, features, train_tokens, max_cap, vocab_size, data):
    while True:
        for img, caption in data.items():
            if img in features:
                feature = features[img][0]
                input1, input2, output = bulid_sequences(tokenizer, max_cap, vocab_size, feature, caption)
                yield ({"input_image": input1, "input_sequence": input2}, output)



In [ ]:
[[in_img,in_seq],_out_w]=next(datagenrator(tokenizer,f,train_tokens,max_cap,vocab_size,train_tokens))

In [ ]:
#in_img.shape,in_seq.shape,_out_w.shape

In [ ]:
#featuresize=in_img.shape[1]

In [ ]:
def bulidModel(num_fatures,longestcaptions,vocab_size):
  #FeedForward model
  inputs_img=Input(shape=(num_fatures,),name="input_image")
  fe1=Dropout(0.5)(inputs_img)
  fe2=Dense(256,activation='relu')(fe1)
  #LSTM model -text
  inputts_seq=Input(shape=(longestcaptions,), name="input_sequence")
  se1=Embedding(vocab_size,256,mask_zero=True)(inputts_seq)
  se2=Dropout(0.5)(se1)
  se3=LSTM(256, use_cudnn=False)(se2)


  #marge
  marged_model=add([fe2,se3])
  finalmodel=Dense(256,activation='relu')(marged_model)
  outputs=Dense(vocab_size,activation='softmax')(finalmodel)
  model=Model(inputs=[inputs_img,inputts_seq],outputs=outputs)
  model.compile(loss='categorical_crossentropy',optimizer='adam')
  return model

In [ ]:
model=bulidModel(2048,max_cap,vocab_size)


In [ ]:
dataset = tf.data.Dataset.from_generator(
    lambda: datagenrator(tokenizer, f, train_tokens, max_cap, vocab_size, train_tokens),
    output_signature=(
        {
            "input_image": tf.TensorSpec(shape=(None, 2048), dtype=tf.float32),
            "input_sequence": tf.TensorSpec(shape=(None, max_cap), dtype=tf.int32),
        },
        tf.TensorSpec(shape=(None, vocab_size), dtype=tf.float32),
    )
)

In [126]:
#geneartor=datagenrator(tokenizer,f,train_tokens,max_cap,vocab_size,train_tokens)
model.fit(dataset,epochs=10,steps_per_epoch=len(train_tokens),verbose=1)
model.save('model.keras')

Epoch 1/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - loss: 2.1171
Epoch 2/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - loss: 2.0338
Epoch 3/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - loss: 1.9686
Epoch 4/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - loss: 1.9076
Epoch 5/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - loss: 1.8515
Epoch 6/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 24s 13ms/step - loss: 1.8160
Epoch 7/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - loss: 1.7848
Epoch 8/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - loss: 1.7306
Epoch 9/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - loss: 1.6825
Epoch 10/10
1839/1839 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - loss: 1.6604



##Testing##
```
```



In [2]:
def imgfeature_extraction(path,model):
  try:
      img=Image.open(path)

      img=img.resize((299,299))
      img=np.expand_dims(img,axis=0)
      img=img/127.5
      img=img-1.0
      feature=model.predict(img)

      return feature
  except:
    return None


In [3]:
def getword(indx,tokenizer):
  return list(tokenizer.word_index)[indx-1]

In [4]:
def genearte_caption(model,tokenizer,imgFeature,longestcaptions):
  in_text='start'
  for i in range(longestcaptions):
    seq=tokenizer.texts_to_sequences([in_text])[0]
    seq=pad_sequences([seq],maxlen=longestcaptions,padding='post')
    yhat=model.predict([imgFeature,seq],verbose=0)
    indx=np.argmax(yhat)
    word_index=getword(indx,tokenizer)
    if word_index=='end':
      break
    in_text+=' '+word_index
  return in_text[6:]

In [5]:
cnn_model=Xception(include_top=False,pooling='avg')
img='/content/1.jpg'
imgFeature=imgfeature_extraction(img,cnn_model)
imgFeature

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


array([[0.06515392, 0.        , 0.02060204, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [6]:
cap_model=load_model('/content/model.keras')


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/trainer.py:210: UserWarning: Model doesn't support `jit_compile=True`. Proceeding with `jit_compile=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
#pickle.dump(tokenizer,open('/content/tokenizer.bin','wb'))

NameError: name 'tokenizer' is not defined

In [8]:
tokenizer=pickle.load(open('/content/tokenizer.bin','rb'))


In [10]:
genearte_caption(cap_model,tokenizer,imgFeature,33)



'brown dog is running through the grass'